# Predicting House Sale Prices
housing data for the city of Ames, Iowa,

In [1]:
import numpy as np
from sklearn.metrics import mean_squared_error

from sklearn import linear_model
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import KFold
import seaborn as sns
%matplotlib inline

In [2]:
data = pd.read_csv('AmesHousing.tsv',delimiter='\t')

In [3]:
def manage_housing_data(df):
    #change the collumns of years to better
    #first count the new value
    #second delete wrong values
    #third delete exsisting columns
    
    years_sold = df['Yr Sold'] - df['Year Built']
    df['Years Before Sale'] = years_sold
    df.drop(df.index[df['Years Before Sale'] < 0], inplace = True)

    years_since_remod = df['Yr Sold'] - df['Year Remod/Add']
    df['Years_since_remod'] = years_since_remod
    df.drop(df.index[df['Years_since_remod'] < 0], inplace = True)
 
    df = df.drop(["Year Built", "Year Remod/Add"], axis = 1)
    
    #delete unnessesary fields
    df = df.drop(["Sale Condition", "Sale Type", "PID", "Order"], axis=1)

    return(df)

In [4]:
def transform_features(df, drop = 0.2):
    
    #all to numeric    
    df=df.apply(pd.to_numeric, errors='ignore')
    
    #remove, if too many nulls
    nulls = df.isnull().sum()
    drops = nulls[(nulls > int(len(df)*drop))]

    df = df.drop(drops.index, axis=1)

    #numeric fields formating
    nulls = df.select_dtypes(include=['int', 'float']).isnull().sum()
    #cols = nulls[(nulls < int(len(df)*drop)) & (nulls > 0)]
    cols = nulls[(nulls > 0)]
    
    #loop all columns and set median to NaN values
    for i in cols.index:
        m_value=df[i].median()
        df[i]=df[i].fillna(m_value)    
        
    #remove fields with empty strings
    empty_strings = df.select_dtypes(include=['object']).isnull().sum()
    drops = empty_strings[empty_strings > 0]
    df = df.drop(drops.index, axis=1)
    
    return(df)

In [5]:
def select_features(df, target, nominal_numeric, corr_limit = 0.3, unique_limit = 10):
    
    #step set categories
    cols = []
    
    for col in nominal_numeric:
        if col in df.columns:
            cols.append(col)
    
    text_cols = df.select_dtypes(include=['object']).columns.tolist()
    cols = cols + text_cols
    
    #select columns having enough unique value counts to be transformed
    unique = df[cols].apply(lambda col: len(col.value_counts()))
    
    #drop others
    drops = unique[unique > unique_limit].index
    df = df.drop(drops, axis=1)
    
    #remaining columns
    cols = [item for item in cols if item not in drops]
    
    #columns to object
    for col in cols:
        df[col] = df[col].astype('object')
    
    #text cols to dummies
    
    text_cols = df.select_dtypes(include=['object']).columns

    for col in text_cols:
        col_dummies = pd.get_dummies(df[col],prefix=[col])
        df = pd.concat([df, col_dummies], axis=1)
        del df[col]
    df.reset_index(inplace=True, drop=True)
    
    #step correlation management
    df_corr = df.select_dtypes(include=['integer', 'float'])
    corrmat = df_corr.corr()
    sorted_corrs = corrmat[target].abs().sort_values()
    weak_corrs = sorted_corrs[sorted_corrs < corr_limit]
    df = df.drop(weak_corrs.index, axis=1)
    
    
    return(df)



In [6]:
def train_and_test(df,target, k):
    
    train = df[:1460]
    test = df[1460:]
    
    
    features = df.columns.drop(target)
    lr = linear_model.LinearRegression()

    if k == 0:
        train = df[:1460]
        test = df[1460:]
        lr.fit(train[features], train[target])
        predictions = lr.predict(test[features])
        mse = mean_squared_error(test[target], predictions)
        rmse = np.sqrt(mse)
    
    if k == 1:
        # Randomize  rows (frac=1)
        shuffled_df = df.sample(frac=1, )
        train = df[:1460]
        test = df[1460:]
        
        lr.fit(train[features], train[target])
        predictions_one = lr.predict(test[features])        
        
        mse_one = mean_squared_error(test[target], predictions_one)
        rmse_one = np.sqrt(mse_one)
        
        lr.fit(test[features], test[target])
        predictions_two = lr.predict(train[features])        
       
        mse_two = mean_squared_error(train[target], predictions_two)
        rmse_two = np.sqrt(mse_two)
        
        avg_rmse = np.mean([rmse_one, rmse_two])
        print(rmse_one)
        print(rmse_two)
        return avg_rmse

    if k == 2:
        kf = KFold(n_splits=k, shuffle=True)
        rmse_values = []
        for train_index, test_index, in kf.split(df):
            train = df.iloc[train_index]
            test = df.iloc[test_index]
            lr.fit(train[features], train[target])
            predictions = lr.predict(test[features])
            mse = mean_squared_error(test[target], predictions)
            rmse = np.sqrt(mse)
            rmse_values.append(rmse)
        print(rmse_values)
        avg_rmse = np.mean(rmse_values)
        return avg_rmse


In [9]:
target = "SalePrice"

nominal_numeric = ["MS SubClass", "Alley", "Mas Vnr Type",
                    "Garage Type","Misc Feature","Sale Type","Sale Condition"]

new_df=manage_housing_data(data)
new_df=transform_features(new_df)
features_df = select_features(new_df, target, nominal_numeric, corr_limit = 0.25)

a=train_and_test(features_df,target,2)
print("avg_rmse ", a)

[27925.959352700702, 32463.409371622114]
avg_rmse  30194.68436216141


In [8]:
features_df

,Lot Frontage,Lot Area,Overall Qual,Mas Vnr Area,BsmtFin SF 1,Total Bsmt SF,1st Flr SF,2nd Flr SF,Gr Liv Area,Bsmt Full Bath,...,['Foundation']_PConc,['Heating QC']_Ex,['Heating QC']_TA,['Central Air']_N,['Central Air']_Y,['Kitchen Qual']_Ex,['Kitchen Qual']_Gd,['Kitchen Qual']_TA,['Paved Drive']_N,['Paved Drive']_Y
0,141.0,31770,6,112.0,639.0,1080.0,1656,0,1656,1.0,...,0,0,0,0,1,0,0,1,0,0
1,80.0,11622,5,0.0,468.0,882.0,896,0,896,0.0,...,0,0,1,0,1,0,0,1,0,1
2,81.0,14267,6,108.0,923.0,1329.0,1329,0,1329,0.0,...,0,0,1,0,1,0,1,0,0,1
3,93.0,11160,7,0.0,1065.0,2110.0,2110,0,2110,1.0,...,0,1,0,0,1,1,0,0,0,1
4,74.0,13830,5,0.0,791.0,928.0,928,701,1629,0.0,...,1,0,0,0,1,0,0,1,0,1
5,78.0,9978,6,20.0,602.0,926.0,926,678,1604,0.0,...,1,1,0,0,1,0,1,0,0,1
6,41.0,4920,8,0.0,616.0,1338.0,1338,0,1338,1.0,...,1,1,0,0,1,0,1,0,0,1
7,43.0,5005,8,0.0,263.0,1280.0,1280,0,1280,0.0,...,1,1,0,0,1,0,1,0,0,1
8,39.0,5389,8,0.0,1180.0,1595.0,1616,0,1616,1.0,...,1,1,0,0,1,0,1,0,0,1
9,60.0,7500,7,0.0,0.0,994.0,1028,776,1804,0.0,...,1,0,0,0,1,0,1,0,0,1
